In [1]:
%matplotlib ipympl
from astrotime.encoders.wotan.flatten import flatten
from ipywidgets import interactive
import ipywidgets as widgets
from astrotime.util.series import TSet
from astrotime.loaders.MIT import MITOctavesLoader
import matplotlib.pyplot as plt
from astrotime.config.context import astrotime_initialize
import torch
from hydra import initialize, compose

version = "MIT_period.octaves"
overrides = []
initialize(version_base=None, config_path="../../../config")
cfg = compose(config_name=version, overrides=overrides)
cfg.platform.gpu = -1
device: torch.device = astrotime_initialize(cfg, version+".plot")
sector = cfg.data.sector_range[0]
element = 100
cfg.data['nharmonics'] = 6
cfg.data['noctaves'] = 6

In [3]:
data_loader = MITOctavesLoader(cfg.data)
data_loader.initialize(TSet.Train)
data_element = data_loader.get_dataset_element( sector, data_loader.TICS(sector)[element] )
[time, flux] = [data_element.data_vars[k].values for k in ['time', 'y'] ]

Loading TESS data from archive.stsci.edu...


In [4]:
def func(method, window_length, break_tolerance, edge_cutoff='0.1', cval=5):
    f, ax = plt.subplots(2, sharex=True, figsize=(12, 12))
    if method == 'trim_mean' or method == 'winsorize':
        cval /= 10  # must be a fraction >0, <0.5
        if cval >=0.5:
            cval = 0.49
    time1, flux1, flatten_lc, trend_lc = flatten(
        time,
        flux,
        method=method,
        window_length=window_length,
        edge_cutoff=edge_cutoff,
        break_tolerance=break_tolerance,
        cval=cval
        )
    ax[0].plot(time1, trend_lc, color='black', linewidth=3)
    ax[0].scatter(time1, flux1, edgecolors='k', c='yellow', s=30)
    ax[0].set_xlim(min(time1), max(time1))
    ax[0].set_ylabel('Raw flux')
    ax[1].scatter(time1, flatten_lc, edgecolors='k', c='black', s=30)
    ax[1].set_ylim(0.995, 1.005)
    ax[1].set_ylabel('Detrended flux')
    plt.xlabel('Time (days)')
    f.subplots_adjust(hspace=0)
    plt.show();
    return time

In [5]:
y1=interactive(
    func,
    method=["biweight", "hodges", "welsch", "andrewsinewave", "winsorize"],
    window_length=(0.1, 2, 0.1),
    break_tolerance=(0, 1, 0.1),
    edge_cutoff=(0, 1, 0.1),
    cval=(1, 9, 1)
    )
y2=interactive(
    func,
    method=["hspline", "pspline", "rspline"],
    window_length=(0.1, 2, 0.1),
    break_tolerance=(0, 1, 0.1),
    edge_cutoff=(0, 1, 0.1),
    cval=(1, 9, 1)
    )
widgets.VBox([widgets.HBox([y1, y2])])